<a href="https://colab.research.google.com/github/CristianoDataScience/Classificador_Spam/blob/main/modelo_Spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando Bibliotecas

In [24]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sys

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

## Importando Base de Dados

In [3]:
spam = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spam.csv')
spam.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
spam.shape

(5572, 2)

In [5]:
spam['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [6]:
# Vetorizando a coluna 'Message'
previ = spam['Message']
classe = spam['Category']
vetorizador = TfidfVectorizer()
previsores = vetorizador.fit_transform(previ)
print(previsores.shape)

(5572, 8709)


In [7]:
print(vetorizador.get_feature_names_out()[10:100])

['0207' '02072069400' '02073162414' '02085076972' '021' '03' '04' '0430'
 '05' '050703' '0578' '06' '07' '07008009200' '07046744435' '07090201529'
 '07090298926' '07099833605' '07123456789' '0721072' '07732584351'
 '07734396839' '07742676969' '07753741225' '0776xxxxxxx' '07781482378'
 '07786200117' '077xxx' '078' '07801543489' '07808' '07808247860'
 '07808726822' '07815296484' '07821230901' '078498' '07880867867'
 '0789xxxxxxx' '07946746291' '0796xxxxxx' '07973788240' '07xxxxxxxxx' '08'
 '0800' '08000407165' '08000776320' '08000839402' '08000930705'
 '08000938767' '08001950382' '08002888812' '08002986030' '08002986906'
 '08002988890' '08006344447' '0808' '08081263000' '08081560665' '0825'
 '083' '0844' '08448350055' '08448714184' '0845' '08450542832'
 '08452810071' '08452810073' '08452810075over18' '0870' '08700435505150p'
 '08700469649' '08700621170150p' '08701213186' '08701237397' '08701417012'
 '08701417012150p' '0870141701216' '087016248' '08701752560'
 '087018728737' '087024118271

### Modelo de Treino e Teste RandomForestClassifier

In [8]:
X_treinamento, X_teste, y_treinamento,y_teste = train_test_split(previsores, classe, test_size=0.3)

In [9]:
floresta = RandomForestClassifier(n_estimators=1000)
floresta.fit(X_treinamento,y_treinamento)

RandomForestClassifier(n_estimators=1000)

In [10]:
previsoes = floresta.predict(X_teste)
print(previsoes)

['ham' 'ham' 'ham' ... 'ham' 'spam' 'ham']


### Matriz de confusão

In [11]:
# Gerando matriz de confusão
print(confusion_matrix(y_teste, previsoes))

[[1433    0]
 [  44  195]]


## Modelo de Classificação e Acurácia 
* Métricas
* Acurácia

In [12]:
print(accuracy_score(y_teste, previsoes))
print(metrics.classification_report(y_teste, previsoes))

0.9736842105263158
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1433
        spam       1.00      0.82      0.90       239

    accuracy                           0.97      1672
   macro avg       0.99      0.91      0.94      1672
weighted avg       0.97      0.97      0.97      1672



In [13]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(spam['Category'])
print(y)

[0 0 1 ... 0 0 0]


In [14]:
mensagens = spam['Message'].values
X_train,X_test,y_train,y_test = train_test_split(mensagens, y, test_size=0.3)
print(X_train)

['Actually fuck that, just do whatever, do find an excuse to be in tampa at some point before january though'
 '&lt;DECIMAL&gt; m but its not a common car here so its better to buy from china or asia. Or if i find it less expensive. I.ll holla'
 'Shall i come to get pickle' ... "I wonder if you'll get this text?"
 'Ok lor thanx... Ü in school?'
 'Ma head dey swell oh. Thanks for making my day']


## Vetorizando os modelos 

In [15]:
vetorizadores = CountVectorizer()
vetorizadores.fit(X_train)
X_train = vetorizadores.transform(X_train)
X_test = vetorizadores.transform(X_test)
print(X_train.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## Criando o modelo Sequecial "Keras"

In [16]:
modelo = Sequential()
modelo.add(Dense(units=10,activation='relu',input_dim=X_train.shape[1]))
modelo.add(Dropout(0.1))
modelo.add(Dense(units=8,activation='relu'))
modelo.add(Dropout(0.1))
modelo.add(Dense(units=1,activation='sigmoid'))

In [17]:
modelo.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                72890     
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 72,987
Trainable params: 72,987
Non-trainable params: 0
_________________________________________________________________


In [ ]:
modelo.fit(X_train, y_train, epochs=20, batch_size=10, verbose=True, validation_data=(X_test, y_test))

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Reshape:0", shape=(None, 10), dtype=float32), dense_shape=Tensor("gradient_tape/sequential/dense/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


390/390 [==============================] - 3s 5ms/step - loss: 0.0937 - accuracy: 0.9331 - val_loss: 0.0235 - val_accuracy: 0.9749
Epoch 2/20
390/390 [==============================] - 2s 4ms/step - loss: 0.0144 - accuracy: 0.9887 - val_loss: 0.0192 - val_accuracy: 0.9773
Epoch 3/20
390/390 [==============================] - 2s 4ms/step - loss: 0.0059 - accuracy: 0.9956 - val_loss: 0.0185 - val_accuracy: 0.9779
Epoch 4/20
390/390 [==============================] - 2s 4ms/step - loss: 0.0039 - accuracy: 0.9962 - val_loss: 0.0183 - val_accuracy: 0.9791
Epoch 5/20
390/390 [==============================] - 2s 5ms/step - loss: 0.0032 - accuracy: 0.9969 - val_loss: 0.0174 - val_accuracy: 0.9797
Epoch 6/20
390/390 [==============================] - 2s 4ms/step - loss: 0.0023 - accuracy: 0.9982 - val_loss: 0.0178 - val_accuracy: 0.9809
Epoch 7/20
390/390 [==============================] - 1s 4ms/step - loss: 0.0024 - accuracy: 0.9979 - val_loss: 0.0183 - val_accuracy: 0.9791
Epoch 8/20
390/39

In [19]:
loss, accuracy = modelo.evaluate(X_test, y_test)
print('Loss ', loss)
print('Acurácia ', accuracy)


53/53 [==============================] - 1s 1ms/step - loss: 0.2531 - accuracy: 0.2033
Loss  0.253083199262619
Acurácia  0.2033492773771286


In [20]:
nova_previsao = modelo.predict(X_test)
print(nova_previsao)

[[0.51340383]
 [0.5006554 ]
 [0.50002784]
 ...
 [0.50541353]
 [0.5041364 ]
 [0.4998879 ]]


In [21]:
prev = (nova_previsao > 0.5)
print(prev)

[[ True]
 [ True]
 [ True]
 ...
 [ True]
 [ True]
 [False]]


In [23]:
cm = confusion_matrix(y_test, prev)
print(cm)

[[ 151 1300]
 [  32  189]]
